In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import time
import datetime
import tensorflow as tf
from pandas import read_csv
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils

from sklearn.metrics import mean_squared_error, r2_score

from pandas import read_csv
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras import optimizers

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import chart_studio.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)

import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

# 데이터 임포트

In [2]:
# 데이터 폴더 경로
path = './차량/'

table = pd.DataFrame()

arrs = os.listdir(path)

fnames = []
for arr in arrs:
    fnames.append(arr)
    
for i in range(len(fnames)): 
    data = pd.read_csv(path+fnames[i], 
                        skiprows=1, 
                        names=['TRANSDATE','ENCACPLATE','INDATE','INGATE','OUTDATE','OUTGATE'])
    table = table.append(data)

    rawdata = table.copy()

In [3]:
rawdata

,TRANSDATE,ENCACPLATE,INDATE,INGATE,OUTDATE,OUTGATE
0,2019-03-23,797CE2CAEE45C70CEE67819315A46BDAFF8768CE,2019-03-23 09:35:00,북2,2019-03-23 19:07:00,북2
1,2019-03-14,EB53D4316364B613585EAC6750A3CB4E3D6CD89F,2019-03-14 19:47:00,동2,2019-03-14 22:34:00,남2
2,2019-03-12,ED615B82CEFEA6A80C296F9FC74087AB3CE5326F,2019-03-12 09:08:00,동2,2019-03-12 09:20:00,동2
3,2019-03-22,9852646408556E485A99BDBA899E4D9B0A8EFC15,2019-03-22 09:06:00,북2,NaN,NaN
4,2019-03-31,D83724650064150339B8CC49313719E5CB5590F2,2019-03-31 08:10:00,동2,2019-03-31 09:05:00,동2
...,...,...,...,...,...,...
250761,2019-06-30,F0CC252A2B9CBF0DCD12CA258832A62F020E1B38,2019-06-30 22:52:00,동2,NaN,NaN
250762,2019-06-16,E8C82FE9EB1D9364FFD09FE150F41FBA14019872,2019-06-16 16:50:00,서2,NaN,NaN
250763,2019-06-13,BBBD2C508A6C8AA6F03875B8689627A1A9340D7A,2019-06-13 09:55:00,동2,NaN,NaN
250764,2019-06-15,CABEF04DC64325AB4CEE7E1AD88D87E3CCC978F2,2019-06-15 09:50:00,남4,NaN,NaN


# 데이터 전처리

In [4]:
# indate가 Nan인 행 제거 
rawdata = rawdata.dropna(subset=['INDATE'])

# outdate가 Nan인 행 제거 
rawdata = rawdata.dropna(subset=['OUTDATE'])
rawdata

# OUTDATE 외의 컬럼 값이 모두 Nan인 행 제거
rawdata = rawdata.dropna(thresh=5)  
rawdata

,TRANSDATE,ENCACPLATE,INDATE,INGATE,OUTDATE,OUTGATE
0,2019-03-23,797CE2CAEE45C70CEE67819315A46BDAFF8768CE,2019-03-23 09:35:00,북2,2019-03-23 19:07:00,북2
1,2019-03-14,EB53D4316364B613585EAC6750A3CB4E3D6CD89F,2019-03-14 19:47:00,동2,2019-03-14 22:34:00,남2
2,2019-03-12,ED615B82CEFEA6A80C296F9FC74087AB3CE5326F,2019-03-12 09:08:00,동2,2019-03-12 09:20:00,동2
4,2019-03-31,D83724650064150339B8CC49313719E5CB5590F2,2019-03-31 08:10:00,동2,2019-03-31 09:05:00,동2
5,2019-03-17,073EE207340811F4043AE1D1C6FE4D314AD64FE3,2019-03-17 14:01:00,남3,2019-03-17 15:38:00,남3
...,...,...,...,...,...,...
250363,2019-06-30,E2977BA015330E3207104858205027C1DBF3FF3E,2019-06-30 18:41:00,북2,2019-06-30 18:42:00,북2
250364,2019-06-01,F1EF8DA9B4F127DF5356E4BDBB0ADF15541CD1D7,2019-06-01 06:00:00,북2,2019-06-01 09:48:00,북2
250365,2019-06-22,D532F75628DBF987755A5529C62390B89BA08C7A,2019-06-22 19:35:00,동2,2019-06-22 23:40:00,동2
250366,2019-06-29,70955B7A6A6DE405F1163C02B725A32755BD6AF1,2019-06-29 13:47:00,남2,2019-06-29 15:30:00,북2


In [5]:
# 필요한 입차, 출차시간만 남김
df = rawdata[['INDATE','OUTDATE']]
df.reset_index(inplace=True)
del df['index']

#이용객수 카운트를 위한 컬럼 생성
df['cnt'] = 1

#컬럼 INDATE를 날짜, 시간으로 나누기
cols = ['INDATES','INTIME']
df['temp'] = df['INDATE'].str.split(' ')

for i, c in enumerate(cols):
    df[c] = df['temp'].apply(lambda x: x[i])
    
#컬럼 OUTDATE를 날짜, 시간으로 나누기
cols2 = ['OUTDATES','OUTTIME']
df['temp'] = df['OUTDATE'].str.split(' ')

for i, c in enumerate(cols2):
    df[c] = df['temp'].apply(lambda x: x[i])
    
cols3 = ['INYEAR','INMON','INDAY']
cols4 = ['OUTYEAR', 'OUTMON', 'OUTDAY']

#컬럼 INDATES를 날짜, 시간으로 나누기
df['temp'] = df['INDATES'].str.split('-')
for i, c in enumerate(cols3):
    df[c] = df['temp'].apply(lambda x: x[i])
    
#컬럼 OUTDATES를 날짜, 시간으로 나누기
df['temp'] = df['OUTDATES'].str.split('-')
for i, c in enumerate(cols4):
    df[c] = df['temp'].apply(lambda x: x[i])
    
#컬럼 INTIME에서 시간 추출
df['INHOUR'] = df['INTIME'].apply(lambda x: x.split(':')[0])

#컬럼 OUTTIME에서 시간 추출
df['OUTHOUR'] = df['INTIME'].apply(lambda x: x.split(':')[0])

#df에서 필요한 컬럼만 남김
df = df[['INDATES','INYEAR', 'INMON', 'INDAY', 'INHOUR', 'OUTDATES', 'OUTYEAR', 'OUTMON', 'OUTDAY', 'OUTHOUR', 'INDATE', 'OUTDATE', 'cnt']]

In [6]:
df.to_csv('올림픽.csv', encoding='utf-8-sig')

In [7]:
df

,INDATES,INYEAR,INMON,INDAY,INHOUR,OUTDATES,OUTYEAR,OUTMON,OUTDAY,OUTHOUR,INDATE,OUTDATE,cnt
0,2019-03-23,2019,03,23,09,2019-03-23,2019,03,23,09,2019-03-23 09:35:00,2019-03-23 19:07:00,1
1,2019-03-14,2019,03,14,19,2019-03-14,2019,03,14,19,2019-03-14 19:47:00,2019-03-14 22:34:00,1
2,2019-03-12,2019,03,12,09,2019-03-12,2019,03,12,09,2019-03-12 09:08:00,2019-03-12 09:20:00,1
3,2019-03-31,2019,03,31,08,2019-03-31,2019,03,31,08,2019-03-31 08:10:00,2019-03-31 09:05:00,1
4,2019-03-17,2019,03,17,14,2019-03-17,2019,03,17,14,2019-03-17 14:01:00,2019-03-17 15:38:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6655888,2019-06-30,2019,06,30,18,2019-06-30,2019,06,30,18,2019-06-30 18:41:00,2019-06-30 18:42:00,1
6655889,2019-06-01,2019,06,01,06,2019-06-01,2019,06,01,06,2019-06-01 06:00:00,2019-06-01 09:48:00,1
6655890,2019-06-22,2019,06,22,19,2019-06-22,2019,06,22,19,2019-06-22 19:35:00,2019-06-22 23:40:00,1
6655891,2019-06-29,2019,06,29,13,2019-06-29,2019,06,29,13,2019-06-29 13:47:00,2019-06-29 15:30:00,1


# 전체 일별 라인 그래프

In [8]:
line = pd.DataFrame(df['cnt'].groupby(df['INDATES']).sum())
line

,cnt
INDATES,
2018-01-01,4240
2018-01-02,6473
2018-01-03,6453
2018-01-04,6440
2018-01-05,6639
...,...
2020-10-27,5150
2020-10-28,5485
2020-10-29,5445


In [9]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 일별 이용객"

ln = line.iplot(kind='line', layout=layout, xTitle='Hour', yTitle = '차량', color='blue', asFigure=True)
ln

In [10]:
ln.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_1_1.png")

line_json = ln.to_json()
line_json
json_obj = json.loads(line_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d

with open('./13/img_1_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

In [11]:
# file_path = "./sample.json"

# with open(file_path, 'w') as outfile:
#     json.dump(json_obj_d, outfile)

# 시간별

In [12]:
hour = pd.DataFrame(df['cnt'].groupby(df['INHOUR']).sum())
hour

,cnt
INHOUR,
00,13934
01,8835
02,15964
03,6813
04,12154
05,165718
06,262976
07,311997
08,801494


In [13]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 시간별 차량 이용수"

bar = hour.iplot(kind='bar', layout=layout, xTitle='Hour', yTitle = '차량', color='skyblue', asFigure=True)
bar

In [14]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_2_1.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_2_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 월별

In [15]:
mon = pd.DataFrame(df['cnt'].groupby(df['INMON']).sum())
mon

,cnt
INMON,
01,559284
02,467555
03,534049
04,565189
05,634980
06,574006
07,574413
08,603051
09,593273


In [16]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 월별 차량 이용수"

bar = mon.iplot(kind='bar', layout=layout, xTitle='MONTH', yTitle = '차량', color='darkslateblue', asFigure=True)
bar

In [17]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_3_1.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_3_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 연도별

In [18]:
year = pd.DataFrame(df['cnt'].groupby(df['INYEAR']).sum())
year

,cnt
INYEAR,
2018,2539043
2019,2728821
2020,1388029


In [19]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 연도별 차량 이용수"

bar = year.iplot(kind='bar', layout=layout, xTitle='YEAR', yTitle = '차량', color='blue', asFigure=True)
bar

In [20]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_2_2.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_2_2.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 계절별

In [21]:
sea = df['cnt'].groupby(df['INMON']).sum()

In [22]:
se = ['겨울','봄','여름','가을']

In [23]:
sea = pd.DataFrame(sea)

In [24]:
sea

,cnt
INMON,
01,559284
02,467555
03,534049
04,565189
05,634980
06,574006
07,574413
08,603051
09,593273


In [25]:
sea.reset_index(inplace=True)

In [26]:
win = (sea['cnt'][0] + sea['cnt'][1] + sea['cnt'][11]) / 3
spr = (sea['cnt'][2] + sea['cnt'][3] + sea['cnt'][4]) / 3
summ = (sea['cnt'][5] + sea['cnt'][6] + sea['cnt'][7]) / 3 
aut = (sea['cnt'][8] + sea['cnt'][9] + sea['cnt'][10]) / 3 

In [27]:
season_list = []
season_list.append(win)
season_list.append(spr)
season_list.append(summ)
season_list.append(aut)

In [28]:
table = pd.DataFrame(season_list)
table.index = se

In [29]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 계절별 차량 이용수"

bar = table.iplot(kind='bar', layout=layout, xTitle='Season', yTitle = '차량', color='powderblue', asFigure=True)
bar

In [30]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_3_2.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_3_2.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)